$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\cset}[1]{\mathcal{#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}
\newcommand{\ip}[3]{\left<#1,#2\right>_{#3}}
\newcommand{\given}[]{\,\middle\vert\,}
\newcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}
\newcommand{\grad}[]{\nabla}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
$$

# Part 2: Summary Questions
<a id=part2></a>

This section contains summary questions about various topics from the course material.

You can add your answers in new cells below the questions.

**Notes**

- Clearly mark where your answer begins, e.g. write "**Answer:**" in the beginning of your cell.
- Provide a full explanation, even if the question doesn't explicitly state so. We will reduce points for partial explanations!
- This notebook should be runnable from start to end without any errors.

### CNNs

1. Explain the meaning of the term "receptive field" in the context of CNNs.
Answer: A receiptive field of an output pixel is the region of the input image that influences it. In order for the model to properly generalize, the reciptive field needs to be the whole input image, or at least most of it.

2. Explain and elaborate about three different ways to control the rate at which the receptive field grows from layer to layer. Compare them to each other in terms of how they combine input features.

Answer:
Here's three different ways to control the rate at which the receptive field grows from layer to layer:
1. kernel size: increasing the kernel size causes the receptive field of the pixel at the next layer to be bigger.
2. stride: stride changes how much area is covered by the convolution, so it increases the receptive field from layer to layer
3. dilation: the dilation defines a spacing between the weights and allows the neuron to "see" a larger region of the input space
The first method capture more input features, but it also increases the number of parameters that will need to be learned by the model, as the weight size is bigger.
Dilation is increasing the receptive field without capturing more input features at each stage (some input values are multiplied by zero). The number of parameters in the model is not increasing either.
The stride also is increasing the receptive field without capturing more input features. The number of parameters in the model is not increasing either.

3. Imagine a CNN with three convolutional layers, defined as follows:

In [1]:
import torch
import torch.nn as nn

cnn = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5, stride=2, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=7, dilation=2, padding=3),
    nn.ReLU(),
)

cnn(torch.rand(size=(1, 3, 1024, 1024), dtype=torch.float32)).shape

torch.Size([1, 32, 122, 122])

What is the size (spatial extent) of the receptive field of each "pixel" in the output tensor?

Answer:
We will calculate this by working backwards from the final layer.
conv 16->32:
this is a layer with a kernel size of 7,a dilation of 2, and a stride of 1 as such the effective kernel size is 2*7-1 = 13.
Hence the receptive field w.r.t previous layer is 1*1 + (13-1) = 13
maxPool2d:
this layer can be thought of as a kerne with size 2 and stride 2.
as such the receptive field is 2*13 + (2-2) = 26
conv 4->16:
This layer has a kernel size of 5 and a stride of 2, so the receptive field is:
2*26 + (5-2) = 55
maxPool2d: the receptive field is again multiplied by 2 and grows to 110.
conv 3->4: this has a kernel size of 3 and stride of 1 and as such the receptive field is:
1*110 + (3-1) = 112.
so the final size is 112x112.


4. You have trained a CNN, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$, and $f_l(\cdot;\vec{\theta}_l)$ is a convolutional layer (not including the activation function).

  After hearing that residual networks can be made much deeper, you decide to change each layer in your network you used the following residual mapping instead $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)+\vec{x}$, and re-train.

  However, to your surprise, by visualizing the learned filters $\vec{\theta}_l$ you observe that the original network and the residual network produce completely different filters. Explain the reason for this.

Answer:
The difference in the learned filters between the original and residual network is due to the introduction of the skip connection in residual network. It allows direct propagation of information across layers, which is different from the original structure where each layer must learn to extract the most relevant information from the input without knowledge of higher level features that might be useful later.
As a result, the residual network can learn complementary residual features that can mitigate the vanishing gradient problem in very deep networks.


### Dropout

1. Consider the following neural network:

In [2]:
import torch.nn as nn

p1, p2 = 0.1, 0.2
nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Dropout(p=p1),
    nn.Dropout(p=p2),
)

Sequential(
  (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Dropout(p=0.1, inplace=False)
  (3): Dropout(p=0.2, inplace=False)
)

If we want to replace the two consecutive dropout layers with a single one defined as follows:
```python
nn.Dropout(p=q)
```
what would the value of `q` need to be? Write an expression for `q` in terms of `p1` and `p2`.

Answer:
To replace the two consecutive dropout layers with a single one, we need to find the drop probability q that would have the same effect as applying p1 and p2 consecutively. We can use for this the probability formula for independent events:
p(A or B) = p(A) + p(B) - p(A and B)
The probability of a dropout for a specific neuron would be:
q = p1 + p2 - p1*p2
With the numbers of the previous question, we would get q=0.28

2. **True or false**: dropout must be placed only after the activation function.

Answer:
False, dropout can be applied before or after the activation function. Placing dropout after the activation function because the function introduces non-linearities into the network and can help the dropout operation to have a stronger regularizing effect. However, applying dropout before the activation function can reduce the computation of the activation function because some neurons are already zero.

3. After applying dropout with a drop-probability of $p$, the activations are scaled by $1/(1-p)$. Prove that this scaling is required in order to maintain the value of each activation unchanged in expectation.

Answer:
After applying dropout with a drop-probability of $p$, the activations are scaled by $1/(1-p)$. This scaling is required in order to maintain the value of each activation unchanged in expectation.
Here's why:
When we apply dropout to an activation layer, we either drop it with a probability of p or keep it with a probability of (1-p).
The expected value of the activation is:
E(a) = a * (1-p) + 0 * p = a(1-p)

If we scale by $1/(1-p)$ this expression, the expected value will be unchanged after applying the dropout:
a(1-p)/(1-p) = a
After this scaling, the drop cells will remain 0 and the non dropped will keep the same value.

### Losses and Activation functions

1. You're training a an image classifier that, given an image, needs to classify it as either a dog (output 0) or a hotdog (output 1). Would you train this model with an L2 loss? if so, why? if not, demonstrate with a numerical example. What would you use instead?

Answer:
No, we would not train this model with an L2 loss.
The L2 loss (MSE), is a regression loss function that measures the average squared distance (difference) between the predicted and target value.

The common use for L2 loss is problems where the output is a continuous e.g: numeric value or a probability. BUT, in the given task the output has only two possible values (dog or hotdog).

As stated before, L2 loss returns a continues number, assuming this is the probability distribution over the classes. A true label for an image can be either a dog or a hotdog with a so the distribution is [0, 1] or [1,0], but the L2 loss might return [0.5, 0.5].
In this case, the L2 loss would be (0.5-0)^2+(0.5-1)^2=0.5.
This does not reflect the fact that the model is uncertain and doesn't strongly predict either class.

A better loss function would be binary cross-entropy loss, which is common in binary classification.
The log loss function, measures the difference between two probability distributions: predicted prob. and true prob.

In the example, when the output is [0.5, 0.5], the binary cross-entropy loss would penalize the model for being uncertain and not strongly predicting the true class.

The loss defined as:

L = -[ylog(p) + (1-y)log(1-p)]
where p is the predicted probability of the positive class (hotdog) and y is ground-truth label.

2. After months of research into the origins of climate change, you observe the following result:

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/de/PiratesVsTemp%28en%29.svg/1200px-PiratesVsTemp%28en%29.svg.png?20110518040647" /></center>

You decide to train a cutting-edge deep neural network regression model, that will predict the global temperature based on the population of pirates in `N` locations around the globe.
You define your model as follows:

In [3]:
import torch.nn as nn

N = 42  # number of known global pirate hot spots
H = 128
mlpirate = nn.Sequential(
    nn.Linear(in_features=N, out_features=H),
    nn.Sigmoid(),
    *[
        nn.Linear(in_features=H, out_features=H), nn.Sigmoid(),
    ]*24,
    nn.Linear(in_features=H, out_features=1),
)

While training your model you notice that the loss reaches a plateau after only a few iterations.
It seems that your model is no longer training.
What is the most likely cause?

Answer:
The sigmoid function is usually a bad choice for networks with a very large depth since it tends to have very small gradients (the maximum possible one is at 0 which is 0.25). this means that after 24 layers the gradients would have a magnitude of 1/4^24 which is very small and as such the network will suffer from vanishing gradients and not converge after only a few iterations.

3. Referring to question 2 above: A friend suggests that if you replace the `sigmoid` activations with `tanh`, it will solve your problem. Is he correct? Explain why or why not.

Answer:
The maximum gradient of tanh is 1 so in theory it is possible to use it to avoid vanishing gradients but this forces all inputs be 0 which is not practical.
In practice if we want to have a non trivial network using tanh then it would suffer from the same vanishing gradients as with the sigmoid.

4. Regarding the ReLU activation, state whether the following sentences are **true or false** and explain:
  1. In a model using exclusively ReLU activations, there can be no vanishing gradients.
  1. The gradient of ReLU is linear with its input when the input is positive.
  1. ReLU can cause "dead" neurons, i.e. activations that remain at a constant value of zero.

Answer:
1. In a model using exclusively ReLU activations, there can be no vanishing gradients.
False: In a model using exclusively ReLU activations, there still can be vanishing gradients in a deep network. For example, if the input values to ReLU are negatives, the gradient would be 0 which could lead to vanishing gradients.
2. The gradient of ReLU is linear with its input when the input is positive.
True: When the input is positive, ReLU is returning the same value so the gradient will be 1. That's mean that the gradient is linear with its input.
3. ReLU can cause "dead" neurons, i.e. activations that remain at a constant value of zero.
True: ReLU can cause "dead" neurons if the input to ReLU are always less or equal to zero. It can happen if the input is negative and the convolution still output negatives numbers, or if the previous layer has a large negative bias term that will always output negative numbers. In such case, the gradient fails and the weights stop being updated on this path.

### Optimization

1. Explain the difference between: stochastic gradient descent (SGD), mini-batch SGD and regular gradient descent (GD).

Answer:

Regular Gradient Descent (GD) computes the gradient of the loss function with respect to the model parameters using the entire dataset. It then updates the model parameters by taking a step proportional to the negative gradient, multiplied by a learning rate. GD can be computationally expensive and is not suitable for large datasets.

Stochastic Gradient Descent (SGD) takes a different approach by updating the model parameters after processing each individual training example. As a result, the updates are much more frequent compared to GD, but can be noisy due to the random variations in the training examples. However, the noisy updates can help SGD escape local optima and find better solutions. SGD is faster than GD in terms of convergence because it makes more frequent updates, but it can have high variance in the parameter updates.

Mini-Batch SGD is a compromise between GD and SGD. Instead of using the entire dataset or just one example, Mini-Batch SGD computes the gradient and updates the parameters based on a small subset (mini-batch) of the dataset. Mini-Batch SGD is a balance between the computational efficiency of GD and the faster convergence of SGD. It reduces the variance of updates compared to SGD and is less noisy than SGD, but it still updates the parameters more frequently compared to GD so it's less expensive than GD, and also help escape from local optima.

In summary, GD is computationally expensive but has low variance, SGD is faster but has high variance, and Mini-Batch SGD is a compromise between the two, balancing computational efficiency and convergence speed.

2. Regarding SGD and GD:
  1. Provide at least two reasons for why SGD is used more often in practice compared to GD.
  2. In what cases can GD not be used at all?

Answer:
    1.) Here's 3 reasons for why SGD is used more often in practice compared to GD:
        - Computational memory: SGD requires less memory than GD because it runs only one example at a time before updating the parameters. It also need fewer computation per iteration. It's more scalable to use SGD for large dataset.
        - Avoid local optima: SGD random updates can escape the model from local optima and find better solutions. It allows the model to explore different regions of the parameter space and potentially escape from some local optima
        - Faster convergence: SGD update more quickly than GD because it needs fewer computation per iteration (only one example VS whole dataset). It can lead to faster convergence of the model during training.
    2.) GD can't be used at all when the dataset is too large to fit into memory for one iteration, or when the model is too complex to compute the gradient on the entire dataset. SGD or mini-batch SGD are more appropriate in these cases.

3. You have trained a deep resnet to obtain SoTA results on ImageNet.
While training using mini-batch SGD with a batch size of $B$, you noticed that your model converged to a loss value of $l_0$ within $n$ iterations (batches across all epochs) on average.
Thanks to your amazing results, you secure funding for a new high-powered server with GPUs containing twice the amount of RAM.
You're now considering to increase the mini-batch size from $B$ to $2B$.
Would you expect the number of of iterations required to converge to $l_0$ to decrease or increase when using the new batch size? explain in detail.

Answer:

If the batch size is increased from B to 2B in a model trained with mini-batch SGD, the number of iterations required to converge is likely to decrease.

Indeed, with a larger batch size, the gradient estimate used for updating the parameters is based on a larger sample of the dataset. This can lead to a more accurate estimate of the true gradient, as the noise introduced by individual examples in smaller batch sizes is reduced.

As a result, increasing the batch size from B to 2B can result in more stable and consistent updates, which can help the model converge faster.

However, larger batch sizes may result in smaller updates to the model parameters, which can reduce the amount of exploration in the parameter space and potentially affect the model's ability to escape from local optima. It's worth noticing that the loss value of $l_0$ might not be found by less than n iterations if the model remain in local optima for more than n iterations.

4. For each of the following statements, state whether they're **true or false** and explain why.
  1. When training a neural network with SGD, every epoch we perform an optimization step for each sample in our dataset.
  1. Gradients obtained with SGD have less variance and lead to quicker convergence compared to GD.
  1. SGD is less likely to get stuck in local minima, compared to GD.
  1. Training  with SGD requires more memory than with GD.
  1. Assuming appropriate learning rates, SGD is guaranteed to converge to a local minimum, while GD is guaranteed to converge to the global minimum.
  1. Given a loss surface with a narrow ravine (high curvature in one direction): SGD with momentum will converge more quickly than Newton's method which doesn't have momentum.

Answer:
For the following questions, we will assume that SGD refers to mini-batch SGD. It's what we understood from a piazza answer from an instructor.
1. When training a neural network with SGD, every epoch we perform an optimization step for each sample in our dataset
False: SGD computes the gradient and updates the parameters based on a small subset (mini-batch) of the dataset, not for each sample. However, if the batch size used is 1, it can perform an optimization step for each sample.
2. Gradients obtained with SGD have less variance and lead to quicker convergence compared to GD.
False: Gradients obtained with SGD have more variance because they are computed on a smaller portion of the dataset (GD compute on the whole). It can lead to slower convergence compared to GD, but allows larger exploration in the parameter space and potentially find better solutions.
3. SGD is less likely to get stuck in local minima, compared to GD.
True: SGD is less likely to get stuck in local minima, compared to GD, because it performs updates on a mini-batch of examples which introduces more randomness in the optimization process.
4. Training with SGD requires more memory than with GD.
False: Training with SGD requires less memory than with GD. GD needs to keep the entire dataset in memory in order to computer the gradient at each iteration, while SGD needs to keep only the mini batch (assuming that the mini-batch size is smaller than the dataset size).
5. Assuming appropriate learning rates, SGD is guaranteed to converge to a local minimum, while GD is guaranteed to converge to the global minimum.
False: SGD and GD could not be guaranteed to converge to a local or global minimum. Assuming appropriate learning rates, GD is guaranteed to converge to the global minimum only if the loss function has a single global minimum. Otherwise, GD can get stuck in local minima, saddle points, or other suboptimal points. SGD introduces randomness in the updates, which can help it escape local minima and find better solutions compared to GD. However, this randomness also makes it less deterministic and harder to converge to the exact optimum. SGD is not guaranteed to converge to a local or global minimum, as it can continue to fluctuate around the optimal solution.
6. Given a loss surface with a narrow ravine (high curvature in one direction): SGD with momentum will converge more quickly than Newton's method which doesn't have momentum.
True: SGD with momentum will converge more quickly than Newton's method which doesn't have momentum. Using momentum with SGD reduce the zig-zag oscillations that would happen with regular SGD and allows it to move faster in the direction of the gradient. Newton's method is a second-order method which calculate the descent direction on the loss function. It will be hard for it to adapt on a narrow ravine. As a result, SGD with momentum is more suitable in this case.

5. In tutorial 5 we saw an example of bi-level optimization in the context of deep learning, by embedding an optimization problem as a layer in the network.
  **True or false**: In order to train such a network, the inner optimization problem must be solved with a descent based method (such as SGD, LBFGS, etc).
  Provide a mathematical justification for your answer.
    False: Even though the outer problem is solved with a gradient based optimizer and as such must be differentiable, we only calculate the hessian w.r.t a constant value of y and as such do not need it to be differentiable and can calculate it using any chosen method,
    we only need to differentiate f at the chosen y.

6. You have trained a neural network, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$ for some arbitrary parametrized functions $f_l(\cdot;\vec{\theta}_l)$.
  Unfortunately while trying to break the record for the world's deepest network, you discover that you are unable to train your network with more than $L$ layers.
7. Explain the concepts of "vanishing gradients", and "exploding gradients".
    vanishing gradients: gradients early in the network get very small values and approach 0, this is usually due to depth, choice of activation and normalization.
    exploding gradients: gradients get extremely large values and grow out of control, this can be fixed by capping gradients and normalizing.
8. How can each of these problems be caused by increased depth?
    Answer: The nature of the chaing rule is to multiply the gradients of each layer, this means that if they have norms of less or more than one then heir sizes will grow exponentially w.r.t the number of layers.
9. Provide a numerical example demonstrating each.
    vanishing gradients: in a network where each layer implements 0.5*sin(x), the output of 0 would be 0 but every layers gradient would be 0.5 w.r.t previous layer. this means that gradients will decay exponentially
    exploding gradients: in a network where each layer implements 2*sin(x), the output of 0 would be 0 but every layers gradient would be 2 w.r.t previous layer. this means that gradients will explode exponentially
10. Assuming your problem is either of these, how can you tell which of them it is without looking at the gradient tensor(s)?
    Answer: vanishing gradients cause very small changes in the weight after every optimization step while exploding gradients cause huge changes that completely change the network.
    thus by looking at how different he network is after a step we can deduce the problem.

### Backpropagation

1. You wish to train the following 2-layer MLP for a binary classification task:
  $$
  \hat{y}^{(i)} =\mat{W}_2~ \varphi(\mat{W}_1 \vec{x}^{(i)}+ \vec{b}_1) + \vec{b}_2
  $$
  Your wish to minimize the in-sample loss function is defined as
  $$
  L_{\mathcal{S}} = \frac{1}{N}\sum_{i=1}^{N}\ell(y^{(i)},\hat{y}^{(i)}) + \frac{\lambda}{2}\left(\norm{\mat{W}_1}_F^2 + \norm{\mat{W}_2}_F^2 \right)
  $$
  Where the pointwise loss is binary cross-entropy:
  $$
  \ell(y, \hat{y}) =  - y \log(\hat{y}) - (1-y) \log(1-\hat{y})
  $$
  
  Write an analytic expression for the derivative of the final loss $L_{\mathcal{S}}$ w.r.t. each of the following tensors: $\mat{W}_1$, $\mat{W}_2$, $\mat{b}_1$, $\mat{b}_2$, $\mat{x}$.

Answer:

We will use the chain rule principle to calculate the derivative of the final loss w.r.t. each vector:

$$\frac{\partial L_{\mathcal{S}}}{\partial W_{2}}=
\sum_{i=1}^m \frac{\partial L_{\mathcal{S}}}{\partial  \hat{y}^{(i)}} \frac{\partial \hat{y}^{(i)}}{\partial \vec{z}^{(i)}_2} \frac{\partial \vec{z}^{(i)}_2}{\partial W_{2}} = \sum_{i=1}^m \frac{1}{m} \cdot \frac{\hat{y}^{(i)} - y^{(i)}}{\hat{y}^{(i)} (1 - \hat{y}^{(i)})} \cdot a_1^{(i)} = \frac{1}{m} \sum_{i=1}^m (\hat{y}^{(i)} - y^{(i)}) a_1^{(i)}
$$


$$\frac{\partial L_{\mathcal{S}}}{\partial \vec{b}1} = \sum_{i=1}^m \frac{\partial L{\mathcal{S}}}{\partial \hat{y}^{(i)}} \frac{\partial \hat{y}^{(i)}}{\partial \vec{z}^{(i)}_2} \frac{\partial \vec{z}^{(i)}_2}{\partial \vec{a}^{(i)}_1} \frac{\partial \vec{a}^{(i)}_1}{\partial \vec{z}^{(i)}_1} \frac{\partial \vec{z}^{(i)}_1}{\partial \vec{b}1} \\ = \sum_{i=1}^m \frac{1}{m} \cdot \frac{\hat{y}^{(i)} - y^{(i)}}{\hat{y}^{(i)} (1 - \hat{y}^{(i)})} \cdot W_2^\top \hat{y}^{(i)} (1 - \hat{y}^{(i)}) \cdot \sigma'(\vec{z}1^{(i)}) = \frac{1}{m} \sum_{i=1}^m (\hat{y}^{(i)} - y^{(i)}) \cdot W_2^\top \cdot \sigma'(\vec{z}_1^{(i)})
$$

2. The derivative of a function $f(\vec{x})$ at a point $\vec{x}_0$ is
  $$
  f'(\vec{x}_0)=\lim_{\Delta\vec{x}\to 0} \frac{f(\vec{x}_0+\Delta\vec{x})-f(\vec{x}_0)}{\Delta\vec{x}}
  $$
  
3. Explain how this formula can be used in order to compute gradients of neural network parameters numerically, without automatic differentiation (AD).
    Answer: we can iterate through the parameters of the network and for each one, do a small change to its value and calculate the numerical expression for it.
    this would give approximations for each entry of the gradient which combined will result in the approximated gradient.
  
4. What are the drawbacks of this approach? List at least two drawbacks compared to AD.
    speed: we are required to run the entire network for each parameter in it which could result in millions of runs for just one step.
    precision: this method requires working with extremely small numbers which results in very unstable calculations and many errors.

3. Given the following code snippet:
  1. Write a short snippet that implements that calculates gradient of `loss` w.r.t. `W` and `b` using the approach of numerical gradients from the previous question.
  2. Calculate the same derivatives with autograd.
  3. Show, by calling `torch.allclose()` that your numerical gradient is close to autograd's gradient.

In [1]:
import torch

N, d = 100, 5
dtype = torch.float64
X = torch.rand(N, d, dtype=dtype)
W, b = torch.rand(d, d, requires_grad=True, dtype=dtype), torch.rand(d, requires_grad=True, dtype=dtype)

def foo(W, b):
    return torch.mean(X @ W + b)

loss = foo(W, b)
print(f"{loss=}")

def get_num_grad(W, b):
    eps = 1e-5 # standard value
    with torch.no_grad():
        grad_W = W.clone()
        grad_b = b.clone()
        for i in range(len(W.view(-1))):
            params_W = W.clone().reshape(-1)
            # set upper point
            params_W[i] = W.view(-1)[i] + eps
            upper_val = foo(params_W.view(W.size(0), -1), b)
            # set lower point
            params_W[i] = W.view(-1)[i] - eps
            lower_val = foo(params_W.view(W.size(0), -1), b)
            grad_W.view(-1)[i] = (upper_val - lower_val)/ (2*eps)
        for i in range(len(b.view(-1))):
            params_b = b.clone()
            # set upper point
            params_b.view(-1)[i] = b.view(-1)[i] + eps
            upper_val = foo(W, params_b)
            # set lower point
            params_b.view(-1)[i] = b.view(-1)[i] - eps
            lower_val = foo(W, params_b)
            grad_b.view(-1)[i] = (upper_val - lower_val)/ (2*eps)
    return grad_W, grad_b


# TODO: Calculate gradients numerically for W and b
grad_W, grad_b = get_num_grad(W, b)


# TODO: Compare with autograd using torch.allclose()
loss.backward()
autograd_W = W.grad
autograd_b = b.grad
assert torch.allclose(grad_W, autograd_W)
assert torch.allclose(grad_b, autograd_b)


loss=tensor(1.7246, dtype=torch.float64, grad_fn=<MeanBackward0>)


### Sequence models

1. Regarding word embeddings:
2. Explain this term and why it's used in the context of a language model.
    Answer: A word embedding is a vectorized representation of a word. the purpose of such embeddings in language models is to embed words in a continuous space where coordinated have semantic meaning and as such allow models to perform calculations on them.
3. Can a language model like the sentiment analysis example from the tutorials be trained without an embedding (i.e. trained directly on sequences of tokens)? If yes, what would be the consequence for the trained model? if no, why not?
    Answer: Most language models only operate on vectorized data and as such would require some form of embedding even as simple as a one hot encoding.
    Some models like a simple distance based bag of words model can operate without embedding but these methods are very limited and achieve quite bad results.


2. Considering the following snippet, explain:
3. What does `Y` contain? why this output shape?
    Answer: for each integer in the range (0-42) we learn a unique vector that acts as its embedding (of size 42,000). Y contains the vectors corresponding to each entry of x.
4. How you would implement `nn.Embedding` yourself using only torch tensors.
    I would one hot encode the integers and then use some form of transformation to cause the vectors to be nearly orthogonal in a non trivial way, such as a DFT or some form of positional encoding.

In [5]:
import torch.nn as nn

X = torch.randint(low=0, high=42, size=(5, 6, 7, 8))
embedding = nn.Embedding(num_embeddings=42, embedding_dim=42000)
Y = embedding(X)
print(f"{Y.shape=}")

Y.shape=torch.Size([5, 6, 7, 8, 42000])


3. Regarding truncated backpropagation through time (TBPTT) with a sequence length of S: State whether the following sentences are **true or false**, and explain.
4. TBPTT uses a modified version of the backpropagation algorithm.
    True: TBPTT runs backpropagation on chunks of size S of input but does run it on the entire network to avoid running gradients through the entire depth of the sequence
5. To implement TBPTT we only need to limit the length of the sequence provided to the model to length S.
    False: TBPTT does not limit the length of the sequence but rather truncates the paths of the gradient calculation and as such no length limit is required.
3. TBPTT allows the model to learn relations between input that are at most S timesteps apart.
    False: even though gradient calculations only occur on sub-sequences of length S, the model processes the entire sequence and holds a global context for its entirety.

### Attention

1. In tutorial 7 (part 2) we learned how to use attention to perform alignment between a source and target sequence in machine translation.
2. Explain qualitatively what the addition of the attention mechanism between the encoder and decoder does to the hidden states that the encoder and decoder each learn to generate (for their language). How are these hidden states different from the model without attention?
    Answer: attention allows for the encoder and decoder to consider a global context and as such draw connections between words that are far away in the sequence, additionally it requires the embeddings generated by the encoder and decoder to work in a way s.t a simple attention mechanism can rate their similarity and as such it pushes the two latent spaces closer together.
  
3. After learning that self-attention is gaining popularity thanks to the shiny new transformer models, you decide to change the model from the tutorial: instead of the queries being equal to the decoder hidden states, you use self-attention, so that the keys, queries and values are all equal to the encoder's hidden states (with learned projections). What influence do you expect this will have on the learned hidden states?
    Answer: doing this would be quite similar to the attention mechanism of a transformer since RNNs remember the last word they saw much better than the previous ones and as such it would allow the RNN to draw global connections on the input data. It would not however allow for learning a distinct decoder latent space that can globally attend to the encoder outputs and as such restrict the decoder's capacity.


### Unsupervised learning

1. As we have seen, a variational autoencoder's loss is comprised of a reconstruction term and  a KL-divergence term. While training your VAE, you accidentally forgot to include the KL-divergence term.
What would be the qualitative effect of this on:

2. Images reconstructed by the model during training ($x\to z \to x'$)?
    The KL divergence term ensures that the distribution of output images is similar to that of the input images, not having this term would allow the model to overfit much better and reconstruct images with more precision but only those from the training set. other images will suffer from this and will either be of poor quality or have artifacts from images in the training set.
3. Images generated by the model ($z \to x'$)?
    Generated images would have the same problems and either be of very low quality or be very similar to training images.

2. Regarding VAEs, state whether each of the following statements is **true or false**, and explain:
3. The latent-space distribution generated by the model for a specific input image is $\mathcal{N}(\vec{0},\vec{I})$.
    False: the model has a loss term which encourages these distributions to be similar but it doesn't make them equal.
4. If we feed the same image to the encoder multiple times, then decode each result, we'll get the same reconstruction.
    False: VAE's are non-deterministic and as such they sample from distributions as part of their calculation, this means that each run will get different results.
5. Since the real VAE loss term is intractable, what we actually minimize instead is it's upper bound, in the hope that the bound is tight.
    True: we only minimize a proxy for the real loss since the real loss cannot be optimized.

2. Regarding GANs, state whether each of the following statements is **true or false**, and explain:
3. Ideally, we want the generator's loss to be low, and the discriminator's loss to be high so that it's fooled well by the generator.
    True: the discriminator loss relates to how well the discriminator did at determining the nature of it's input. Our goal is to fail the discriminator and as such this should be high.
4. It's crucial to backpropagate into the generator when training the discriminator.
    True: the generator's distribution is changing at every step and as such without getting gradients from the precise calculation used to generate the images, it would be impossible for the discriminator to learn it and to try to detect it.
5. To generate a new image, we can sample a latent-space vector from $\mathcal{N}(\vec{0},\vec{I})$.
    True: the input to a GAN generator is a random noise vector sampled from this distribution.
6. It can be beneficial for training the generator if the discriminator is trained for a few epochs first, so that it's output isn't arbitrary.
    False: the discriminator has to have access to generated images in order to learn to differentiate the distributions since otherwise it is a trivial task.
7. If the generator is generating plausible images and the discriminator reaches a stable state where it has 50% accuracy (for both image types), training the generator more will further improve the generated images.
    False: once this point is reached, continuing to train will not improve the model.